# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
orders = pd.read_csv("Orders.csv")
orders.tail()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
245083,344619,567092,21509,2011,9,5,12,cowboys and indians birthday card,12,2011-09-16 12:48:00,0.42,13196.0,United Kingdom,5.04
245084,344620,567092,21506,2011,9,5,12,"fancy font birthday card,",12,2011-09-16 12:48:00,0.42,13196.0,United Kingdom,5.04
245085,344621,567092,22985,2011,9,5,12,wrap billboard fonts design,25,2011-09-16 12:48:00,0.42,13196.0,United Kingdom,10.50
245086,344622,567092,22568,2011,9,5,12,feltcraft cushion owl,8,2011-09-16 12:48:00,3.75,13196.0,United Kingdom,30.00
245087,344623,567092,22570,2011,9,5,12,feltcraft cushion rabbit,8,2011-09,NaN,NaN,NaN,NaN


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
#aggregation of amount_spent for unique customers
orders_agg = orders.groupby(["CustomerID","Country"]).agg({"amount_spent":"sum"})
orders_agg

,,amount_spent
CustomerID,Country,
12346.0,United Kingdom,77183.60
12347.0,Iceland,2790.86
12348.0,Finland,1487.24
12350.0,Norway,334.40
12352.0,Norway,1561.81
...,...,...
18280.0,United Kingdom,180.60
18281.0,United Kingdom,80.82
18282.0,United Kingdom,100.21


In [5]:
#customers with amount spent above the 95th percentile (aka. 0.95 quantile)
Percentile_95 = orders_agg["amount_spent"].quantile(0.95)
VIP_Customers = orders_agg[orders_agg["amount_spent"] > Percentile_95]
VIP_Customers

,,amount_spent
CustomerID,Country,
12346.0,United Kingdom,77183.60
12409.0,Switzerland,6207.08
12415.0,Australia,102087.88
12428.0,Finland,7548.12
12433.0,Norway,6395.23
...,...,...
18092.0,United Kingdom,7155.52
18102.0,United Kingdom,167112.29
18109.0,United Kingdom,4851.12


In [6]:
#customers ´with amount spent between the 75th and 95th percentile
Preferred_Customers = orders_agg[(orders_agg["amount_spent"] > orders_agg["amount_spent"].quantile(0.75)) & (orders_agg["amount_spent"] < orders_agg["amount_spent"].quantile(0.95))]
Preferred_Customers

,,amount_spent
CustomerID,Country,
12347.0,Iceland,2790.86
12348.0,Finland,1487.24
12352.0,Norway,1561.81
12356.0,Portugal,2753.08
12359.0,Cyprus,3495.73
...,...,...
18245.0,United Kingdom,1672.81
18251.0,United Kingdom,4314.72
18257.0,United Kingdom,1562.87


In [7]:
#add a new column named customers_group to inlcude the VIP customers, prefffered customers and others
orders_agg.loc[orders_agg["amount_spent"].isin(VIP_Customers["amount_spent"]), "customers_group"] = "VIP"
orders_agg.loc[orders_agg["amount_spent"].isin(Preferred_Customers["amount_spent"]), "customers_group"] = "Preferred"
orders_agg["customers_group"] = orders_agg["customers_group"].fillna("Others")
orders_agg

,,amount_spent,customers_group
CustomerID,Country,,
12346.0,United Kingdom,77183.60,VIP
12347.0,Iceland,2790.86,Preferred
12348.0,Finland,1487.24,Preferred
12350.0,Norway,334.40,Others
12352.0,Norway,1561.81,Preferred
...,...,...,...
18280.0,United Kingdom,180.60,Others
18281.0,United Kingdom,80.82,Others
18282.0,United Kingdom,100.21,Others


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [15]:
# Identify the country with the most VIP customers
vip_counts = orders_agg[orders_agg["customers_group"] == "VIP"]
sorting = vip_counts.groupby("Country").count().sort_values(by = "Country", ascending = False)
max_vip_country = sorting.index[0]
max_vip_country

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [29]:
vip_plus_preferred_customers = orders_agg[(orders_agg["customers_group"] == "VIP") | (orders_agg["customers_group"] == "Preferred")]
result = (vip_plus_preferred_customers.groupby(["Country", "customers_group"]).size()  # Count the number of rows in each group
    .reset_index(name="count")  # Convert the Series to a DataFrame with a "count" column
    .sort_values(by="count", ascending=False)  # Sort by the "count" column in descending order
)

print(result)

            Country customers_group  count
39   United Kingdom       Preferred    588
40   United Kingdom             VIP    143
15          Germany       Preferred     29
13           France       Preferred     17
3           Belgium       Preferred      7
14           France             VIP      7
37      Switzerland       Preferred      6
16          Germany             VIP      6
5   Channel Islands       Preferred      4
33            Spain       Preferred      4
30         Portugal       Preferred      3
27           Norway       Preferred      3
22            Japan       Preferred      3
0         Australia       Preferred      3
8           Denmark       Preferred      3
11          Finland       Preferred      3
7            Cyprus       Preferred      3
10             EIRE             VIP      2
2           Austria       Preferred      2
38      Switzerland             VIP      2
21            Italy       Preferred      2
34            Spain             VIP      2
31         

In [21]:
orders_agg

,,amount_spent,customers_group
CustomerID,Country,,
12346.0,United Kingdom,77183.60,VIP
12347.0,Iceland,2790.86,Preferred
12348.0,Finland,1487.24,Preferred
12350.0,Norway,334.40,Others
12352.0,Norway,1561.81,Preferred
...,...,...,...
18280.0,United Kingdom,180.60,Others
18281.0,United Kingdom,80.82,Others
18282.0,United Kingdom,100.21,Others
